In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('C:/Users/s_csmscox/jupyterSave/fashion-mnist_train.csv')
test = pd.read_csv('C:/Users/s_csmscox/jupyterSave/fashion-mnist_test.csv')

train_label = train['label']
train.drop(['label'], axis=1, inplace=True)

test_label = test['label']
test.drop(['label'], axis=1, inplace=True)

### 정규화
scaler_train = MinMaxScaler()
scaler_train.fit(train)
x_data_train_norm = scaler_train.transform(train)

scaler_test = MinMaxScaler()
scaler_test.fit(train)
x_data_test_norm = scaler_test.transform(test)

### tensorflow 기능을 이용해서 one hot encoding을 생성
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(train_label, depth=10))

In [3]:
## relu

# placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W = tf.get_variable('W1', shape=[784,512],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random.normal([512]))
layer = tf.nn.relu(tf.matmul(X,W) + b)

W2 = tf.get_variable('W2', shape=[512,256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random.normal([256]))
layer2 = tf.nn.relu(tf.matmul(layer,W2) + b2)

W3 = tf.get_variable('W3', shape=[256,128],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random.normal([128]))
layer3 = tf.nn.relu(tf.matmul(layer2,W3) + b3)

W4 = tf.get_variable('W4', shape=[128,10],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random.normal([10]))

logit = tf.matmul(layer3,W4) + b4
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# parameter
# num_of_epoch = 2000
# batch_size = 10000

# # 학습용 함수
# def run_train(sess,train_x, train_t):
#     print('### 학습 시작 ###')
#     # 초기화
#     sess.run(tf.global_variables_initializer())
    
#     for step in range(num_of_epoch):
#         total_batch = int(train_x.shape[0] / batch_size)
        
#         for i in range(total_batch):
#             batch_x = train_x[i*batch_size:(i+1)*batch_size]
#             batch_t = train_t[i*batch_size:(i+1)*batch_size]           
#             _, loss_val = sess.run([train,loss],
#                                    feed_dict={X: batch_x, T: batch_t})
            
#         if step % 500 == 0:
#             print('Loss : {}'.format(loss_val))
#     print('### 학습 종료 ###')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
num_of_epoch = 1000

# Accuracy    
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# K-Fold Cross Validation
cv = 5          # Fold의 수
results = []    
                
kf = KFold(n_splits=cv, shuffle=True) 

for training_idx, validation_idx in kf.split(x_data_train_norm):
    training_x = x_data_train_norm[training_idx] # Fancy indexing
    training_t = t_data_train_onehot[training_idx]
    
    val_x = x_data_train_norm[validation_idx]
    val_t = t_data_train_onehot[validation_idx]
    
    # run_train(sess,training_x,training_t)
    print('### 학습 시작 ###')
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        _, loss_val = sess.run([train,loss],
                                   feed_dict={X: training_x, T: training_t})
    
        if step % 100 == 0:
            print('Loss : {}'.format(loss_val))
    
    acc = sess.run(accuracy, feed_dict={X:val_x, T:val_t})
    print('측정한 각각의 결과값 : {}'.format(acc))
    results.append(acc)


print('최종 K-Fold 교차검증을 사용한 Accuracy : {}'.format(np.mean(results)))

### 학습 시작 ###
Loss : 3.5390403270721436
Loss : 0.6485595107078552
Loss : 0.5515878200531006
Loss : 0.5132061839103699
Loss : 0.4543035626411438
Loss : 0.44442373514175415
Loss : 0.4222090542316437
Loss : 0.41165637969970703
Loss : 0.3950428068637848
Loss : 0.3813610374927521
측정한 각각의 결과값 : 0.8631666898727417
### 학습 시작 ###
Loss : 2.890681028366089
Loss : 0.6592520475387573
Loss : 0.5479825139045715
Loss : 0.5096691250801086
Loss : 0.4587579667568207
Loss : 0.44227269291877747
Loss : 0.4257897138595581
Loss : 0.4045340120792389
Loss : 0.38736262917518616
Loss : 0.39035165309906006
측정한 각각의 결과값 : 0.8604166507720947
### 학습 시작 ###
Loss : 2.8115251064300537
Loss : 0.6634060740470886
Loss : 0.5489011406898499
Loss : 0.47393667697906494
Loss : 0.49306026101112366
Loss : 0.4620361626148224
Loss : 0.4292508661746979
Loss : 0.40142735838890076
Loss : 0.38369491696357727
Loss : 0.3790864944458008
측정한 각각의 결과값 : 0.8544166684150696
### 학습 시작 ###
Loss : 3.0613300800323486
Loss : 0.6494472622871399
Loss 

In [6]:
result = sess.run(H, feed_dict={X: x_data_test_norm})
pred = []

for i in result:
    m = i.max()
    for j in range(10):
        if i[j] == m:
            pred.append(j)
            break

cnt = 0
for predict, answer in zip(pred, test_label):
    if predict == answer:
        cnt += 1

print("Accuracy : {}".format((cnt/len(pred))))

Accuracy : 0.8602


In [7]:
## dropout 추가

# placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W = tf.get_variable('W1d', shape=[784,512],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random.normal([512]))
layer = tf.nn.relu(tf.matmul(X,W) + b)

W2 = tf.get_variable('W2d', shape=[512,256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random.normal([256]))
_layer2 = tf.nn.relu(tf.matmul(layer,W2) + b2)
layer2 = tf.nn.dropout(_layer2, rate=0.5)

W3 = tf.get_variable('W3d', shape=[256,128],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random.normal([128]))
_layer3 = tf.nn.relu(tf.matmul(layer2,W3) + b3)
layer3 = tf.nn.dropout(_layer3, rate=0.3)

W4 = tf.get_variable('W4d', shape=[128,10],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random.normal([10]))

logit = tf.matmul(layer3,W4) + b4
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# parameter
# num_of_epoch = 2000
# batch_size = 10000

# # 학습용 함수
# def run_train(sess,train_x, train_t):
#     print('### 학습 시작 ###')
#     # 초기화
#     sess.run(tf.global_variables_initializer())
    
#     for step in range(num_of_epoch):
#         total_batch = int(train_x.shape[0] / batch_size)
        
#         for i in range(total_batch):
#             batch_x = train_x[i*batch_size:(i+1)*batch_size]
#             batch_t = train_t[i*batch_size:(i+1)*batch_size]           
#             _, loss_val = sess.run([train,loss],
#                                    feed_dict={X: batch_x, T: batch_t})
            
#         if step % 500 == 0:
#             print('Loss : {}'.format(loss_val))
#     print('### 학습 종료 ###')


In [8]:
num_of_epoch = 2000

# Accuracy    
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# K-Fold Cross Validation
cv = 5          # Fold의 수
results = []    
                
kf = KFold(n_splits=cv, shuffle=True) 

for training_idx, validation_idx in kf.split(x_data_train_norm):
    training_x = x_data_train_norm[training_idx] # Fancy indexing
    training_t = t_data_train_onehot[training_idx]
    
    val_x = x_data_train_norm[validation_idx]
    val_t = t_data_train_onehot[validation_idx]
    
    # run_train(sess,training_x,training_t)
    print('### 학습 시작 ###')
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        _, loss_val = sess.run([train,loss],
                                   feed_dict={X: training_x, T: training_t})
    
        if step % 1000 == 0:
            print('Loss : {}'.format(loss_val))
    
    acc = sess.run(accuracy, feed_dict={X:val_x, T:val_t})
    print('측정한 각각의 결과값 : {}'.format(acc))
    results.append(acc)


print('최종 K-Fold 교차검증을 사용한 Accuracy : {}'.format(np.mean(results)))

### 학습 시작 ###
Loss : 4.898796081542969
Loss : 0.4587486982345581
측정한 각각의 결과값 : 0.8513333201408386
### 학습 시작 ###
Loss : 4.5326828956604
Loss : 0.4584708511829376
측정한 각각의 결과값 : 0.8514999747276306
### 학습 시작 ###
Loss : 3.5534863471984863


KeyboardInterrupt: 

In [ ]:
# 공식을 이용해서 직접 img를 흑백처리

from PIL import Image
import matplotlib.pyplot as plt

img = np.asarray(Image.open('C:/Users/s_csmscox/jupyterSave/7.png'))

# 그레이 스케일링
r = 0.2989
g = 0.5870
b = 0.1140
gray = img[:, :, 0] * r + img[:, :, 1] * g + img[:, :, 2] * b
img = Image.fromarray(gray)

# 사이즈 조절
img = img.resize((28,28))

# 예측
img = np.asarray(img)
plt.imshow(img, cmap='gray') # cmap='gray_r의 경우 흑백 반전 = Greys'
plt.show()

img = 255 - img
norm_img = scaler_test.transform(img.reshape(1,-1))

result = sess.run(H, feed_dict={X: norm_img})

for i in result:
    m = i.max()
    for j in range(10):
        if i[j] == m:
            print("예측 값 : {}".format(j))
            break

In [ ]:
result = sess.run(H, feed_dict={X: x_data_test_norm})
pred = []

for i in result:
    m = i.max()
    for j in range(10):
        if i[j] == m:
            pred.append(j)
            break

cnt = 0
for predict, answer in zip(pred, test_label):
    if predict == answer:
        cnt += 1

print("Accuracy : {}".format((cnt/len(pred))))